<a href="https://colab.research.google.com/github/Nasirkhan94/Nasirkhan94/blob/main/Scheduler_Derecta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import heapq
from collections import namedtuple, defaultdict, deque

# Define the structure of a task
Task = namedtuple('Task', ['name', 'experiment', 'duration', 'dependencies', 'independent'])

# Define the experiments and their tasks
experiments = {
    'A': [
        Task('incubation', 'A', 30, [], True),
        Task('cooling', 'A', 20, ['incubation'], True),
    ],
    'B': [
        Task('mixing', 'B', 10, [], False),
        Task('heating', 'B', 40, ['mixing'], True),
        Task('cooling', 'B', 30, ['heating'], True),
    ],
    'C': [
        Task('preparation', 'C', 15, [], False),
        Task('reaction', 'C', 25, ['preparation'], True),
    ],
}

def schedule_tasks(experiments):
    # Initialize the task queue, schedule, and current time
    task_queue = []
    schedule = []
    current_time = 0

    # Dictionary to track dependencies
    dependency_count = defaultdict(int)
    dependents = defaultdict(list)

    for experiment, tasks in experiments.items():
        for task in tasks:
            if not task.dependencies:
                heapq.heappush(task_queue, (current_time, task))
            else:
                dependency_count[task.name] = len(task.dependencies)
                for dep in task.dependencies:
                    dependents[dep].append(task)

    completed_tasks = set()

    while task_queue:
        start_time, task = heapq.heappop(task_queue)
        schedule.append((start_time, task))
        end_time = start_time + task.duration
        completed_tasks.add(task.name)

        # Process the dependents of the completed task
        for dependent_task in dependents[task.name]:
            dependency_count[dependent_task.name] -= 1
            if dependency_count[dependent_task.name] == 0:
                heapq.heappush(task_queue, (end_time, dependent_task))

    return schedule

def print_schedule_and_free_times(schedule):
    # Print the schedule
    print("Schedule:")
    for start_time, task in schedule:
        print(f"At time {start_time}: Start {task.name} of experiment {task.experiment} for {task.duration} minutes (Independent task: {task.independent})")

    # Find free hands times based on independent tasks
    free_hand_times = []
    last_free_end = 0

    for start_time, task in schedule:
        if task.independent:
            free_start = start_time
            free_end = start_time + task.duration
            if free_start > last_free_end:
                free_hand_times.append((last_free_end, free_start))
            last_free_end = free_end

    # Output times when hands are free
    print("\nTimes when hands are free (while tasks are running independently):")
    for start, end in free_hand_times:
        if start != end:
            print(f"From {start} to {end} minutes")

    # Output independent tasks where hands are free
    print("\nIndependent tasks where hands are free:")
    for start_time, task in schedule:
        if task.independent:
            free_start = start_time
            free_end = start_time + task.duration
            print(f"During {task.name} from {free_start} to {free_end} minutes, you can switch to other tasks.")


schedule = schedule_tasks(experiments)

print_schedule_and_free_times(schedule)


Schedule:
At time 0: Start incubation of experiment A for 30 minutes (Independent task: True)
At time 0: Start mixing of experiment B for 10 minutes (Independent task: False)
At time 0: Start preparation of experiment C for 15 minutes (Independent task: False)
At time 10: Start heating of experiment B for 40 minutes (Independent task: True)
At time 15: Start reaction of experiment C for 25 minutes (Independent task: True)
At time 30: Start cooling of experiment A for 20 minutes (Independent task: True)

Times when hands are free (while tasks are running independently):

Independent tasks where hands are free:
During incubation from 0 to 30 minutes, you can switch to other tasks.
During heating from 10 to 50 minutes, you can switch to other tasks.
During reaction from 15 to 40 minutes, you can switch to other tasks.
During cooling from 30 to 50 minutes, you can switch to other tasks.
